In [1]:
%load_ext autoreload
%autoreload 2
import torch
import pandas as pd
import matplotlib.pyplot as plt
from data import DataModule
import numpy as np
import pytorch_lightning as pl
from utils import get_mesh_graph, plot_scalar_field
import os
from lightning_module import FeedForward

Fields = [
    #'Density', 
    'Momentum_x', 
    'Momentum_y', 
    # 'Energy', 
    'Pressure', 
    # 'Temperature', 
    # 'Mach', 
    # 'Pressure_Coefficient'
]
train = True

if os.path.isfile('./stats.csv'):
    stats = pd.read_csv('./stats.csv', index_col=0)
else: 
    stats = None

discretization = ['unstructured', 'structured', 'regular'][2]   
if discretization=='unstructured':
    from model_GNN import *
elif discretization=='regular':
    from model_CNN import *

data_module = DataModule(
    data_dir='./data', batch_size=16, 
    discretization=discretization, 
    interp=None, Fields=Fields, statistics=stats)

# model = ExampleCNN(
#     in_dim=3,
#     out_dim=len(Fields),
# )
model = Gen()


In [2]:


if train:
    model = FeedForward(model)

    trainer = pl.Trainer(gpus=[10], auto_lr_find=True)

    trainer.fit(model, data_module) 
else:
    model = FeedForward.load_from_checkpoint('lightning_logs/version_37/checkpoints/epoch=142-step=5965.ckpt', model=model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
use existing stats
               count           mean          std           min            max
Momentum_x  554400.0     136.093877    30.937971   -178.951724     277.743439
Momentum_y  554400.0       6.475812    19.614627   -118.256022     263.397620
Pressure    554400.0  100407.290471  5555.903614  24928.393217  123880.777893

  | Name  | Type | Params
-------------------------------
0 | model | Gen  | 874 K 
-------------------------------
874 K     Trainable params
0         Non-trainable params
874 K     Total params
3.497     Total estimated model params size (MB)
Epoch 0:   0%|          | 0/21 [00:00<?, ?it/s] /home/aerofoil/anaconda3/envs/fluid/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider in

In [2]:
nodes, edges, elems, marker_dict = get_mesh_graph('./data/ag03/mesh.su2')

In [4]:
nodes.shape

torch.Size([1215, 2])

In [14]:
opt = torch.optim.Adam(model.parameters())
opt.state_dict()['param_groups'][0]['lr']

0.001

In [3]:
df = pd.read_csv('data/ag03/solution_0.2_0.0.csv')

[autoreload of data failed: Traceback (most recent call last):
  File "D:\Users\default.DESKTOP-UVQUUDN\anaconda3\envs\taichi\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "D:\Users\default.DESKTOP-UVQUUDN\anaconda3\envs\taichi\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "D:\Users\default.DESKTOP-UVQUUDN\anaconda3\envs\taichi\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "D:\Users\default.DESKTOP-UVQUUDN\anaconda3\envs\taichi\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "D:\Users\default.DESKTOP-UVQUUDN\anaconda3\envs\taichi\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "D:\Users\default.DESKTOP-UVQUUDN\anaconda3\envs\taichi\li

In [19]:
desc = df.describe().T
desc.loc[Fields]

,count,mean,std,min,25%,50%,75%,max
Density,1215.0,1.223362,0.003868,1.198667,1.221976,1.224801,1.225371,1.245900
Momentum_x,1215.0,82.921717,5.978949,7.041458,79.890335,83.066174,84.704281,96.647028
Momentum_y,1215.0,-0.342209,5.145874,-37.735563,-1.881564,0.002859,1.089882,35.706049
Pressure,1215.0,101188.520998,414.620518,99256.201509,101040.478830,101326.574299,101409.910947,103775.928578


In [4]:
pd.read_csv('./stats.csv', index_col=0)

,count,mean,std,min,max
PointID,554400.0,1.828784,365.303484,0.000000,1369.000000
x,554400.0,0.001545,2.123232,-9.945219,10.000000
y,554400.0,-0.000110,2.107311,-10.000000,10.000000
Density,554400.0,0.003638,0.054438,0.427224,1.435931
Momentum_x,554400.0,0.358260,30.937971,-178.951724,277.743439
Momentum_y,554400.0,0.004070,19.614627,-118.256022,263.397620
Energy,554400.0,771.463065,10979.275871,104963.967817,312642.912122
Pressure,554400.0,300.981793,5555.903614,24928.393217,123880.777893
Temperature,554400.0,0.859663,4.338907,200.938749,308.847638
Mach,554400.0,0.000872,0.087479,0.000101,1.590605


In [8]:
x = data_module.get_example()[0][None]
mask = x[:,0][:, None]

In [44]:
(x * (mask!=0).type_as(x))

tensor([[[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[0.3498, 0.3498, 0.3498,  ..., 0.3498, 0.3498, 0.3498],
          [0.3498, 0.3498, 0.3498,  ..., 0.3498, 0.3498, 0.3498],
          [0.3498, 0.3498, 0.3498,  ..., 0.3498, 0.3498, 0.3498],
          ...,
          [0.3498, 0.3498, 0.3498,  ..., 0.3498, 0.3498, 0.3498],
          [0.3498, 0.3498, 0.3498,  ..., 0.3498, 0.3498, 0.3498],
          [0.3498, 0.3498, 0.3498,  ..., 0.3498, 0.3498, 0.3498]],

         [[0.0122, 0.0122, 0.0122,  ..., 0.0122, 0.0122, 0.0122],
          [0.0122, 0.0122, 0.0122,  ..., 0.0122, 0.0122, 0.0122],
          [0.0122, 0.0122, 0.0122,  ..., 0